In [1]:
!pip uninstall -y tensorflow keras
!pip install tensorflow==2.12 keras==2.12
!pip install pyngrok flask bitsandbytes

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━

In [2]:
NGROK_AUTH_TOKEN ="2rkKQ34C4B2M6b2QKglv5DgePDB_aaMGkmt3kCdTZZbDFs6Y"

In [3]:
import pandas as pd
merged_data = pd.read_csv("/kaggle/input/merge-data-1h-and-1d/merged_data_1d.csv")


from sklearn.ensemble import RandomForestRegressor
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM, Dense, Dropout  
from sklearn.preprocessing import MinMaxScaler  
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def train_model(features,url):
    data = merged_data[features]
    
    if 'close_price' in data.columns:
        X = data.drop('close_price', axis=1)
    else:
        X = data.copy()
    y = merged_data['close_price']
    
    def create_dataset(data, time_step):
        X, y = [], []
        for i in range(len(data) - time_step):
            X.append(data[i:(i + time_step), :])  
            y.append(data[i + time_step, 0]) 
        return np.array(X), np.array(y)
    
    
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    time_step = 7
    
    X, y = create_dataset(data_scaled, time_step)
    
    
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    
    
    
    model = Sequential()
    
    
    model.add(tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.3))  
    
    
    model.add(tf.keras.layers.Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dropout(0.3))
    
    
    model.add(Dense(32, activation='relu'))
    
    
    model.add(Dense(1))  
    
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001) 
    
    
    model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test), 
              callbacks=[early_stopping, reduce_lr])
    
    y_pred = model.predict(X_test)
    
    
    y_pred_rescaled = scaler.inverse_transform(np.concatenate((y_pred, np.zeros((y_pred.shape[0], data.shape[1] - 1))), axis=1))[:, 0]
    y_test_rescaled = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], data.shape[1] - 1))), axis=1))[:, 0]
    

    
    
    mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
    rmse = np.sqrt(mse)  
    mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled) 
    r2 = r2_score(y_test_rescaled, y_pred_rescaled)  
    
    
    direction_actual = np.sign(np.diff(y_test_rescaled))  
    direction_predicted = np.sign(np.diff(y_pred_rescaled))  
    
    
    direction_accuracy = np.mean(direction_actual == direction_predicted)
    
    
    
    evaluation_metrics = {
        'Metric': ['MSE', 'RMSE', 'MAE', 'R2', 'Direction Accuracy'],
        'Value': [mse, rmse, mae, r2, direction_accuracy]
    }
    
    
    df_metrics = pd.DataFrame(evaluation_metrics)
    
    
    
    
    
    dates = merged_data['date'].tail(len(y_test_rescaled)).values  
    
    
    prediction_error = y_test_rescaled - y_pred_rescaled
    
    
    direction_accuracy = np.sign(y_test_rescaled[1:] - y_test_rescaled[:-1]) == np.sign(y_pred_rescaled[1:] - y_pred_rescaled[:-1])
    direction_accuracy = np.concatenate(([np.nan], direction_accuracy)) 
    
    
    results_df = pd.DataFrame({
        'Date': dates,
        'Actual Price': y_test_rescaled,
        'Predicted Price': y_pred_rescaled,
        'Prediction Error': prediction_error,
        'Direction Accuracy': direction_accuracy
    })
    
    model.save(url + '/model.h5')
    results_df.to_csv(url + '/pred.csv', index=False)
    df_metrics.to_csv(url + '/evaluation_metrics.csv', index=False)
    

In [4]:
import torch
from transformers import pipeline, AutoTokenizer
from collections import Counter

# 快取 sentiment pipeline 避免重複加載
sentiment_pipelines = {}

def get_sentiment_pipeline(model_name, device):
    if model_name not in sentiment_pipelines:
        sentiment_pipelines[model_name] = pipeline(
            "sentiment-analysis",
            model=model_name,
            device=device
        )
    return sentiment_pipelines[model_name]

def split_long_text(text, tokenizer, max_tokens=512):
    tokens = tokenizer.tokenize(text)
    chunks = []
    while tokens:
        chunk = tokens[:max_tokens]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
        tokens = tokens[max_tokens:]
    return chunks

def analyze_sentiment(text, model_name, sentiment_map, device, max_length=512):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    text_chunks = split_long_text(text, tokenizer, max_tokens=max_length)

    results = []
    for chunk in text_chunks:
        pipeline_model = get_sentiment_pipeline(model_name, device)
        try:
            result = pipeline_model(chunk, truncation=True, max_length=max_length)[0]
            label = result['label']
            score = result['score']

            if score <= 0.66:
                print(f"[Low confidence] {label} ({score:.2f}) from model '{model_name}'")
                continue

            sentiment_number = sentiment_map.get(label, '0')  # 預設中立
            results.append(sentiment_number)
        except Exception as e:
            print(f"[Error] model {model_name}: {e}")
            continue

    return majority_vote(results) if results else "-9"

def majority_vote(results_list):
    vote_counts = Counter(results_list)
    most_common = vote_counts.most_common()
    max_count = most_common[0][1]
    top_labels = [label for label, count in most_common if count == max_count]
    return '0' if len(top_labels) > 1 else top_labels[0]

def predict_sentiment(text):
    print("=== Start Sentiment Prediction ===")
    models_info = [
        ("ElKulako/cryptobert", {"Bearish": "-1", "Neutral": "0", "Bullish": "1"}),
        ("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", {"positive": "1", "negative": "-1", "neutral": "0"}),
        ("AfterRain007/cryptobertRefined", {"Bullish": "1", "Bearish": "-1", "Neutral": "0"}),
        ("ProsusAI/finbert", {"positive": "1", "negative": "-1", "neutral": "0"})
    ]

    device = 0 if torch.cuda.is_available() else -1
    all_sentiments = []

    for model_name, sentiment_map in models_info:
        print(f"[Model] {model_name}")
        sentiment = analyze_sentiment(text, model_name, sentiment_map, device)
        if sentiment != "-9":
            all_sentiments.append(sentiment)
        else:
            print(f"[Filtered] No confident result from {model_name}")

    print("[Model Sentiments]", all_sentiments)

    if not all_sentiments:
        print("[Warning] All models filtered out, returning Neutral (0)")
        return "0"

    return majority_vote(all_sentiments)


def preload_models():
    print("🚀 預先載入模型中...")
    device = 0 if torch.cuda.is_available() else -1
    model_list = [
        "ElKulako/cryptobert",
        "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
        "AfterRain007/cryptobertRefined",
        "ProsusAI/finbert"
    ]
    for model in model_list:
        _ = get_sentiment_pipeline(model, device)
    print("✅ 所有模型載入完畢")

preload_models()

🚀 預先載入模型中...


config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


✅ 所有模型載入完畢


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

class FinanceLLMAPI:
    def __init__(self, model_name, use_fp16 = True):
        self.model_name = model_name
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.use_fp16 = use_fp16
        self.model = None
        self.tokenizer = None
        self.load_model()

    def load_model(self):
        # 優化的量化配置
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_compute_dtype=torch.float16 if self.use_fp16 else torch.bfloat16,
            bnb_8bit_use_double_quant=True
        )

        # 載入模型
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.float16 if self.use_fp16 else torch.bfloat16,
            low_cpu_mem_usage=True
        )

        # 載入分詞器
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            use_fast=True,
            trust_remote_code=True
        )

        # 設置 pad_token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # 啟用快取
        self.model.config.use_cache = True

    def predict(self, prompt: str, temperature: float = 0.7):
        system_prompt = (
            "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. "
            "Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. "
            "Please ensure that your responses are socially unbiased and positive in nature.\n\n"
            "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. "
            "If you don't know the answer to a question, please don't share false information.\n"
        )
        
        # 你自己的 crypto 專業指令
        crypto_prompt = (
            "You are a professional crypto investment advisor. "
            "Always answer clearly, concisely, and honestly. Focus on cryptocurrency-related insights whenever possible. "
            "If the question is unrelated to crypto, answer it professionally. Limit your response to under 200 words."
        )
        
        # 最終合併成 LLaMA2 指令格式
        full_prompt = f"<s>[INST] <<SYS>>{system_prompt + crypto_prompt}<</SYS>>\n\n{prompt} [/INST]"

        inputs = self.tokenizer(
            full_prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            add_special_tokens=False
        ).to(self.device)


        torch.cuda.empty_cache()

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                do_sample=True,
                temperature=temperature,
                top_k=50,
                top_p=0.95,
                max_new_tokens=1024,
                repetition_penalty=1.1,
                num_return_sequences=1,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 擷取 Advisor 回答內容
        response = decoded
        if "[/INST] " in decoded:
            response = decoded.split("[/INST] ")[-1].strip()
        return response

# 測試
model_name = "AdaptLLM/finance-chat"
llm_api = FinanceLLMAPI(model_name)
response = llm_api.predict("What are the current trends in cryptocurrency?")
print(response)

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


The current trends in cryptocurrency are constantly evolving, but some of the most notable ones include increased adoption by mainstream businesses, regulatory clarity, and technological advancements. We also see a growing interest in decentralized finance (DeFi) and non-fungible tokens (NFTs). Additionally, there has been a surge in blockchain gaming and metaverse applications. However, it's important to note that the market can be volatile, and prices may fluctuate rapidly. As always, it's essential to do your own research and consult with a financial advisor before making any investment decisions.


In [6]:
from flask import Flask, request, jsonify, send_file
import pandas as pd
import threading
import os
from pyngrok import ngrok
import time 


def log_input(input_data):
    print(f"輸入: {input_data}")

def log_output(output_data):
    print(f"輸出: {output_data}")


app = Flask(__name__)
os.makedirs("files", exist_ok=True)

# 你的 ngrok token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(5000).public_url
print(f"🚀 Ngrok 公開網址：{public_url}")

@app.route('/generate_csv', methods=['POST'])
def generate_csv():
    data = request.get_json()
    log_input(data)
    feature = data.get("feature", [])
    model_id = data.get("model_id")
    # 用 model_id 區分檔案
    path = f"files/{model_id}"
    train_model(feature,path)
    path1 = f"files/{model_id}_output1.csv"
    path2 = f"files/{model_id}_output2.csv"
    text = " ".join(feature)
    df1 = pd.DataFrame({"input": [text]})
    df2 = pd.DataFrame({"reversed": [text[::-1]]})

    df1.to_csv(path1, index=False)
    df2.to_csv(path2, index=False)
    results = jsonify({
        "url1": f"{public_url}/download/{model_id}/h5",
        "url2": f"{public_url}/download/{model_id}/csv",
        "url3": f"{public_url}/download/{model_id}/csv2"
    })
    log_output(results)
    return results

@app.route('/download/<model_id>/h5')
def download_h5(model_id):
    path = f"files/{model_id}/model.h5"
    if not os.path.exists(path):
        return jsonify({"error": "File not found"}), 404
    return send_file(path, as_attachment=True)


@app.route('/download/<model_id>/csv')
def download_csv(model_id):
    path = f"files/{model_id}/pred.csv"
    if not os.path.exists(path):
        return jsonify({"error": "File not found"}), 404
    return send_file(path, as_attachment=True)

@app.route('/download/<model_id>/csv2')
def download_csv2(model_id):
    path = f"files/{model_id}/evaluation_metrics.csv"
    if not os.path.exists(path):
        return jsonify({"error": "File not found"}), 404
    return send_file(path, as_attachment=True)



@app.route('/sentiment', methods=['POST'])
def sentiment_api():
    data = request.get_json()
    log_input(data)
    texts = data.get("texts", [])

    if not texts:
        return jsonify({"error": "No texts provided"}), 400

    results = []
    for text in texts:
        sentiment = predict_sentiment(text)
        results.append({"sentiment": sentiment})
    log_output(results)
    return jsonify(results)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json  # 獲取傳來的 JSON 資料
    log_input(data)
    text = data.get("text", "")
    response = llm_api.predict(text)
    log_output(response)
    # 回傳對應結果
    return jsonify({"response": response})

def start_server():
    app.run(port=5000)

threading.Thread(target=start_server).start()

# ✅ 印出可用網址供參考
time.sleep(1)
for i in range(10):
    print(f"✅ POST 請求網址：{public_url}")
    time.sleep(60*60)

🚀 Ngrok 公開網址：https://27ab-34-73-228-105.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off
✅ POST 請求網址：https://27ab-34-73-228-105.ngrok-free.app
輸入: {'texts': ['The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.']}
=== Start Sentiment Prediction ===
[Model] ElKulako/cryptobert
[Low confidence] Bullish (0.62) from model 'ElKulako/cryptobert'
[Filtered] No confident result from ElKulako/cryptobert
[Model] mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
[Model] AfterRain007/cryptobertRefined
[Model] ProsusAI/finbert
[Low confidence] negative (0.63) from model 'ProsusAI/finb

Token indices sequence length is longer than the specified maximum sequence length for this model (3580 > 512). Running this sequence through the model will result in indexing errors


[Low confidence] neutral (0.65) from model 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
[Low confidence] positive (0.64) from model 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
[Low confidence] positive (0.58) from model 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
[Model] AfterRain007/cryptobertRefined


Token indices sequence length is longer than the specified maximum sequence length for this model (3810 > 512). Running this sequence through the model will result in indexing errors


[Model] ProsusAI/finbert


...

[Low confidence] positive (0.57) from model 'ProsusAI/finbert'
[Low confidence] negative (0.40) from model 'ProsusAI/finbert'
[Model Sentiments] ['0', '-1', '0', '0']
輸出: [{'sentiment': '0'}]
輸入: {'text': 'Hello'}
輸出: Hello! As an experienced crypto investment advisor, I would recommend conducting thorough market research before making any investment decisions. It is important to keep up-to-date with current trends and news, analyze market data, and consider factors such as the overall health of the crypto industry, the stability of specific coins, and the potential for future growth. Additionally, it may be beneficial to diversify your portfolio by investing in different types of cryptocurrencies, including stablecoins, utility tokens, and security tokens, to minimize risk and maximize returns.
輸入: {'model_id': '3', 'feature': []}
輸入: {'texts': ['The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitcoin market is too volatile to trust.The Bitc

Token indices sequence length is longer than the specified maximum sequence length for this model (3580 > 512). Running this sequence through the model will result in indexing errors


[Low confidence] neutral (0.65) from model 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
[Low confidence] positive (0.64) from model 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
[Low confidence] positive (0.58) from model 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis'
[Model] AfterRain007/cryptobertRefined


Token indices sequence length is longer than the specified maximum sequence length for this model (3810 > 512). Running this sequence through the model will result in indexing errors


[Model] ProsusAI/finbert
[Low confidence] positive (0.57) from model 'ProsusAI/finbert'
[Low confidence] negative (0.40) from model 'ProsusAI/finbert'
[Model Sentiments] ['0', '-1', '0', '0']
輸出: [{'sentiment': '0'}]
輸入: {'model_id': '1', 'feature': ['close_price', 'S&P 500 Index', 'VIX Volatility Index', 'WTI Crude Oil Futures']}
12/12 [==============================] - 2s 4ms/step
輸出: <Response 200 bytes [200 OK]>
✅ POST 請求網址：https://27ab-34-73-228-105.ngrok-free.app
✅ POST 請求網址：https://27ab-34-73-228-105.ngrok-free.app
✅ POST 請求網址：https://27ab-34-73-228-105.ngrok-free.app
✅ POST 請求網址：https://27ab-34-73-228-105.ngrok-free.app
✅ POST 請求網址：https://27ab-34-73-228-105.ngrok-free.app
✅ POST 請求網址：https://27ab-34-73-228-105.ngrok-free.app
